In [49]:
#PART 1: Connect to a Remote Duet Server
import syft as sy
duet = sy.duet("787cffcb85e058f950b708baf52a362c")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined



KeyboardInterrupt: 

In [17]:
#PART 2: Setting up a Model and our Data
import torch
from torch.utils.data import Dataset, DataLoader,BatchSampler
from preprocess_dl_Input_version4 import *
#from bgru import *
import numpy as np
import pickle
import random
import time
import math
import os
import datetime

In [18]:
config = {"batchSize":64,
          "vector_dim":30,#单个样本的单个token的向量维度
          "maxlen": 900, #单个样本的切片长度
          "dropout":0.4 , #模型dropout的比例
          "lr":0.002,
          "no_cuda":True,
          "seed":42,
          "log_batchsize":100}

In [27]:
#used for remote model
remote_torch = duet.torch

In [20]:
class Masking(sy.Module):
    
    def __init__(self,torch_ref,mask_value=0., *args):
        super(Masking, self).__init__(torch_ref = torch_ref)
        self.supports_masking = True
        self.mask_value = mask_value

    def compute_mask(self, inputs, mask=None):
        a=self.torch_ref.ne(inputs, self.mask_value)
        return a.any(dim=-1, keepdim=False, out=None)
    
    def forward(self, inputs):
        a=self.torch_ref.ne(inputs,self.torch_ref.tensor(self.mask_value))
        boolean_mask = a.any(dim=-1, keepdims=True)
        return inputs * boolean_mask.type(inputs.dtype)

    def get_config(self):
        config = {'mask_value': self.mask_value}
        base_config = super(Masking, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    
class NonMasking(sy.Module):
    
    """Non Masking Layer
    自定义的消除蒙版层，以应对池化不支持蒙版
    """
    
    def __init__(self, torch_ref,**kwargs):
        self.supports_masking = True
        super(NonMasking, self).__init__(torch_ref = torch_ref,**kwargs)
    
    def build(self, input_shape):
        input_shape = input_shape
    
    def compute_mask(self, input, input_mask=None):
        #不传输蒙版
        return None
    
    def forward(self, x, mask=None):
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape

    
class BGRU(sy.Module):    
    def __init__(self,torch_ref):  
        super(BGRU, self).__init__(torch_ref = torch_ref)
        #self.masking = Masking(self.torch_ref,[0.0])
        self.bgru1 = self.torch_ref.nn.GRU(input_size=30, hidden_size=512, num_layers=1, batch_first=False,bidirectional=True)
        self.dropout1 = self.torch_ref.nn.Dropout(p = 0.4, inplace = False)
        self.bgru2 = self.torch_ref.nn.GRU(input_size=1024, hidden_size=512, num_layers=1, batch_first=False,bidirectional=True)
        self.dropout2 = self.torch_ref.nn.Dropout(p = 0.4, inplace= False)
        self.fc =self.torch_ref.nn.Linear(in_features = 1024, out_features = 1, bias = True)
        #self.nonmasking = NonMasking(self.torch_ref)
        #self.kmax = KMaxPooling(1)
        #self.avg = GlobalAveragePooling1D()
        self.kmax=self.torch_ref.nn.AdaptiveMaxPool2d((1,1), return_indices=False) 
        self.avg =self.torch_ref.nn.AdaptiveAvgPool2d(1)
        
        
    def forward(self, inputs,vulner_mask_input):
        x=inputs.view(64,900,30)
        #x = self.masking(x)
        x = self.bgru1(x)
        x = self.dropout1(x[0])
        x = self.bgru2(x)
        x = self.dropout2(x[0])
        x = self.fc(x)
        x = self.torch_ref.sigmoid(x)
        sigmoid = x
        #x = self.nonmasking(x)
        y = vulner_mask_input.view(64,900,900)
        x = self.torch_ref.mul(x, y)
        x = x.view(64,1, 900*900)
        x = self.kmax(x)
        x = self.avg(x)
        return x,sigmoid

In [28]:
# defining the model 
# 这里的local_model也可以加载本地经过初始训练后的模型，只有第一次训练才为下面的代码
local_model=BGRU(torch)#Creating local model    
model = local_model.send(duet) 
print(local_model)
print(model)

In [29]:
#check for cuda 
has_cuda = False
has_cuda_ptr = remote_torch.cuda.is_available()
has_cuda = bool(has_cuda_ptr.get(
    request_block=True,
    reason="To run test locally",
    timeout_secs=5,  # change to something slower
))
use_cuda = not config["no_cuda"] and has_cuda
# now we can set the seed
remote_torch.manual_seed(config["seed"])

device = remote_torch.device("cuda" if use_cuda else "cpu")
    
# if we have CUDA lets send our model to the GPU
if has_cuda:
    model.cuda(device)
else:
    model.cpu()
#---

In [42]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: fe62934d5bd94d699635aae12429b84d>,[dataset_train],,<class 'torch.Tensor'>
1,<UID: b6250e39bbcf4c4cb7f0fd2f6183bdab>,[labels_train],,<class 'torch.Tensor'>
2,<UID: 3cb5ed710366470198884395a551ffe7>,[place_sequence_train],,<class 'torch.Tensor'>
3,<UID: 3ee9b0568e474948879effd4808e81e7>,[train_num],,<class 'syft.lib.python.Int'>


In [43]:
def train(remote_torch,model,weightPath):
    if model.is_local:
        print("Training requires remote model")
        model = model.send(duet)
    else:
        print("remote model")
        model = model  
    model.train()
    dataset_train = duet.store["dataset_train"]
    labels_train = duet.store["labels_train"]
    place_sequence_train = duet.store["place_sequence_train"]
    train_num = duet.store["train_num"]
    iter_num =  train_num.get(request_block=True,reason="size of data set",timeout_secs=5)
    
    #lab=labels_train.get(
     #           reason="To evaluate training progress",
      #          request_block=True,
       #         timeout_secs=5,
        #    )
    #print(len(lab))
    optimizer = remote_torch.optim.Adam(model.parameters(), config["lr"])
    criterion = remote_torch.nn.BCELoss()    
    print("Train...")
    batch_size = config["batchSize"]
    epochs=4
    for epoch in range(epochs):
        nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print("=========="*8 + "%s"%nowtime)
        i=0
        j=0
        running_loss= sy.lib.python.Float(0.0)
        samples=sy.lib.python.Int(0)
        for iter in  range(10):
            j=j+1
            print(batch_size)
            for vp in range(batch_size):
                #print(place_sequence_train)
                vulner_places = remote_torch.diag(place_sequence_train[0*64+vp])
                if vp==0:
                    batched_vulner_places = vulner_places
                    batched_vulner_places=remote_torch.unsqueeze(batched_vulner_places,0)
                    batched_vulner_places=batched_vulner_places.expand(64,900,900)
                else:
                    vulner_places = remote_torch.diag(place_sequence_train[0*64+vp])
                    batched_vulner_places[vp]=vulner_places
            optimizer.zero_grad()
            #iter=0
            outputs,_=model(dataset_train[iter],batched_vulner_places)
            outputs=outputs.view(batch_size)
            #print("outpus:")
            #out=outputs.get(
             # request_block=True,
    #reason="To run test locally",
    #timeout_secs=5,  # change to something slower
     #       )
      #      print(out)
            duet.store.pandas
            train_num = duet.store["labels_train"]
            labels_train[iter]=labels_train[iter].view(batch_size)
            #print("labels")
           # lab=labels_train[iter].get(
            #  request_block=True,
    #reason="To run test locally",
    #timeout_secs=5,  # change to something slower
     #       )
      #      print(lab)
            #计算loss
            loss=criterion(outputs,labels_train[iter])
            #print("loss")
            #print(loss)
            #los=loss.get(
             # request_block=True,
    #reason="To run test locally",
    #timeout_secs=5,  # change to something slower
     #       )
      #      print(los)
                        # 反向传播
            loss.backward()
            # 计算损失并更新权重
            optimizer.step()
            running_loss+= remote_torch.mul(loss.item(),labels_train[iter].shape[0])
            samples +=labels_train[iter] .shape[0]
                        #train_loss = loss_item.resolve_pointer_type()
            local_loss = loss.get(
                reason="To evaluate training progress",
                request_block=True,
                timeout_secs=5,
            )
            #print(local_loss)
            print('batch [{}/{}], Loss: {:.4f}'.format(j,"9", local_loss))
            if i%100 == 0:
                i = i + batch_size
        l = running_loss.get(request_block=True,reason="To evaluate training progress",timeout_secs=5)/samples.get(request_block=True,reason="To evaluate training progress",timeout_secs=5)
        print('Epoch [{}], Loss: {:.4f}'.format(epoch + 1, l))
        #保存模型
    remote_torch.save(model.state_dict(), weightPath)
    print("训练完成...")
    return l

In [44]:
#测试相关函数
def sample_threshold_windows(value_sequence, linetokens, argv):
    """read the output of RNN deaplearning model output sequence and return the classify result
    
    Input the output of RNN model's top activation(sigmoid) layer,
    read them by time step, and judge the value according to the threshold
    value, count the average that value bigger than threshold, and return the
    classify result 1 or 0.
    根据模型的顶层输出窗口读取，窗口内平均值超过阈值则认定为有漏洞
    
    # Arguments
        value_sequence: The output sequence from RNN model
        linetokens: The index of first token each line      // 句首token索引值
        argv: The dict of argument, contain 'k' 'threshold_value'
    
    # Returns
        linenum: The line predict to have vulnerability's line num      // 报为有漏洞的切片行号
    """
        
    #检查参数是否有效
    if "threshold_value" in argv: #argv是参数列表，包括阈值和K值
        threshold_value = argv["threshold_value"]
    else:
        print("Error:Bad threshold value!")
        return -1
    if "k" in argv:
        k = argv["k"]
    else:
        k = 3
        
    #删除尾部重复数值
    value_sequence = list(value_sequence)
   # print('value_sequence',len(value_sequence))##############################################
    vs = len(value_sequence)-1
    while value_sequence[vs] == value_sequence[-1]:
        vs -= 1
    value_sequence = value_sequence[:vs+2]
    #print('value_sequence',len(value_sequence))    ##############################################
    #标注序列的终止位置为序列长度
    for tokenindex in range(len(linetokens)):
        if len(value_sequence) <= linetokens[tokenindex]:
            linetokens = linetokens[:tokenindex]
            linetokens.append(len(value_sequence))
            break
    if len(value_sequence) > linetokens[-1]:
        linetokens.append(len(value_sequence))
   # print('linetokens',len(linetokens))##############################################
    #扫描每一行，输出序列窗口内k-max的平均值大于阈值则认定为有漏洞(对每一行取前K个最大值的平均值，平均值大于阈值就是有漏洞，取该漏洞行行号作为输出结果)
    linenum = []
    for i in range(len(linetokens)-1):
        left = linetokens[i]
        right = linetokens[i+1]
        if left == right:
            right += 1
        window = value_sequence[left: right]
        window = [x[0] for x in window]
        window.sort(reverse = True)
        k_max = window[:k]
        #print("sum(k_max)/ len(k_max)",sum(k_max)/ len(k_max))##################################
        if sum(k_max)/ len(k_max) > threshold_value:
            #如果该行被认定为有漏洞
            linenum.append(i)
    
    return linenum


In [45]:
def generator_of_data1(torch_ref,data, labels, linetokens, vpointers, batchsize, maxlen, vector_dim):
    #输入数据生成器
    iter_num = int(len(data) / batchsize)
    i = 0
    
    while iter_num:
        batchdata = data[i:i + batchsize]
        batched_input = process_sequences_shape(batchdata, maxLen=maxlen, vector_dim=vector_dim)
        batched_labels = labels[i:i + batchsize]
        batched_linetokens = linetokens[i:i + batchsize]
        batched_vpointers = vpointers[i:i + batchsize]
        
        #计算漏洞位置的对角矩阵
        batched_vulner_places = []
        for vp in range(len(batched_vpointers)):
            place_sequence = sample_place_sequence(maxlen, batched_vpointers[vp], batched_linetokens[vp])
            vulner_places = np.diag(place_sequence)     #得到numpy格式的漏洞位置对角矩阵
            batched_vulner_places.append(vulner_places)
        batched_vulner_places = np.array(batched_vulner_places)
   ###########################################################################     
        batched_input=torch_ref.tensor(batched_input,dtype=torch_ref.float32)
        batched_labels=torch_ref.tensor(batched_labels,dtype=torch_ref.float32) 
        batched_vulner_places = torch_ref.tensor(batched_vulner_places,dtype=torch_ref.float32)#zheli  
        
        yield ([batched_input, batched_vulner_places], batched_labels)
        i = i + batchsize
        
        iter_num -= 1
        if iter_num == 0:
            iter_num = int(len(data) / batchsize)
            i = 0

In [46]:

def test(local_model,torch_ref,batch_size, maxlen, vector_dim,weightPath,testdataSet_path,resultPath):
    #测试阶段开始
    #加载模型
 #   print("加载模型")
    #model=BGRU(torch)
    #model.load_state_dict(torch.load(weightPath))
    #local_model.load(weightPath)
 #   if not model.is_local:
#        local_model = model.get(request_block=True,name="model_download",reason="test evaluation",timeout_secs=5)
#    else:
#        local_model = model
#    print("加载模型完成")   
    local_model.eval()
        
    dataset = []
    linetokens = []
    vpointers = []
    funcs = []
    testcase = []
    print("Test...")
    for filename in os.listdir(testdataSet_path):
        if(filename.endswith(".pkl") is False):
            continue
        print(filename)
        f = open(os.path.join(testdataSet_path,filename),"rb")
        #分别是样本向量、句子拆分点、漏洞点、goodbad函数表、句子语料、testcase切片名
        dataset_file, linetokens_file, vpointers_file, funcs_file, corpus_file, testcase_file = pickle.load(f,encoding = 'iso-8859-1')
        f.close()
        dataset += dataset_file
        linetokens += linetokens_file
        vpointers += vpointers_file
        funcs += funcs_file
        testcase +=testcase_file
    print(len(dataset),len(testcase))
    
        #按照漏洞点和所属函数生成0/1标签
    labels = []
    for vp in range(len(vpointers)):
        if vpointers[vp] != []:
            label = 1
            for func in funcs[vp]:
                if "good" in func:
                    label = 0
                    break
        else:
            label = 0   #无漏洞的切片的漏洞点标签为空
        labels.append(label)
        
        #测试模型
    batch_size = 64
    test_generator = generator_of_data1(torch,dataset, labels, linetokens, vpointers, batch_size, maxlen, vector_dim)
    all_test_samples = len(dataset)
    steps_epoch = int(all_test_samples / batch_size)
    #构建测试函数
 #   get_bgru_output = K.function([model.layers[0].input, K.learning_phase()], [model.layers[7].output]) #一层BGRU时读取第5层，两层BGRU时读取第7层，三层BGRU时读取第9层
    #########################################################################################3
    
    TP, TN, FP, FN = 0, 0, 0, 0
    TP_l, TN_l, FP_l, FN_l = 0, 0, 0, 0
    TP_index = []
    results = {}
    dict_testcase2func = {}
    start = time.time()
    #对于每一个样本做测试
    for i in range(steps_epoch):
        print("\r", i, "/", steps_epoch, end="")
        #测试输入
        test_input = next(test_generator)
        #深度学习模型的序列输出
       # layer_output = get_bgru_output([test_input[0][0],0])
        #print("test_input[0][0]",test_input[0][0].shape)
        #print("test_input[0][1]",test_input[0][1].shape)
        _,layer_output = local_model(test_input[0][0])##############################################,test_input[0][1]
        
       
        ###################################################################################
        #测试结果
        for j in range(batch_size):
            index = i*batch_size + j
            #print('\n',testcase[index])
          #  result = sample_threshold_windows(layer_output[0][j], linetokens[index], {'threshold_value':0.5, 'k':1})#result是漏洞行的行号
        ##########################################
         #   print("layer_output",layer_output.shape)
         #   print("layer_output[j]",layer_output[j].shape)
            result = sample_threshold_windows(layer_output[j], linetokens[index], {'threshold_value':0.5, 'k':1})
            #输出统计
            if result:
                y_pred = 1
            else:
                y_pred = 0
            #print("result",result," y_pred", y_pred)  ################################           
        #漏洞分类的指标 & 漏洞分类的函数级指标 & 保存每个测试样本的序列输出
            if y_pred == 0 and labels[index] == 0:
                TN += 1
                TN_l += 1
                with open("BGRU/result_analyze_0.4_k1/TN/"+str(index)+".pkl","wb") as f:
                    pickle.dump(list([layer_output[0][j]]),f)##############################################
                with open("BGRU/result_analyze_0.4_k1/TN_testcase_id_0514.txt",'a+') as ftn:
                    ftn.write(testcase[index]+'\n')
            if y_pred == 0 and labels[index] == 1:
                FN += 1
                FN_l += 1
                with open("BGRU/result_analyze_0.4_k1/FN/"+str(index)+".pkl","wb") as f:
                    pickle.dump(list([layer_output[0][j]]),f)#################################################
                with open("BGRU/result_analyze_0.4_k1/FN_testcase_id_0514.txt",'a+') as ffn:
                    ffn.write(testcase[index]+'\n')
					
            if y_pred == 1 and labels[index] == 0:
                FP += 1
                FP_l += 1
                if not testcase[index].split("/")[0] in dict_testcase2func.keys():
                    dict_testcase2func[testcase[index].split("/")[0]]={}
                for func in funcs[index]:
                    if func in dict_testcase2func[testcase[index].split("/")[0]].keys():
                        dict_testcase2func[testcase[index].split("/")[0]][func].append("FP")
                    else:
                        dict_testcase2func[testcase[index].split("/")[0]][func] = ["FP"]
                with open("BGRU/result_analyze_0.4_k1/FP/"+str(index)+".pkl","wb") as f:
                    pickle.dump(list([layer_output[0][j]]),f)###############################################
                with open("BGRU/result_analyze_0.4_k1/FP_testcase_id_0514.txt",'a+') as ffp:
                    ffp.write(testcase[index]+'\n')					
                    
            if y_pred == 1 and labels[index] == 1:
                TP += 1
                TP_index.append(index)
                flag_l = False
            #基于切片中的漏洞行位置判断是否正确
                for pred in result:
                    if linetokens[index][pred] in vpointers[index]:
                        flag_l = True
                        break
                if flag_l:
                    TP_l += 1
                else:
                    FN_l += 1
                if not testcase[index].split("/")[0] in dict_testcase2func.keys():
                    dict_testcase2func[testcase[index].split("/")[0]]={}
                for func in funcs[index]:
                    if func in dict_testcase2func[testcase[index].split("/")[0]].keys():
                        dict_testcase2func[testcase[index].split("/")[0]][func].append("TP")
                    else:
                        dict_testcase2func[testcase[index].split("/")[0]][func] = ["TP"]
                with open("BGRU/result_analyze_0.4_k1/TP/"+str(index)+".pkl","wb") as f:
                    pickle.dump(list([layer_output[0][j]]),f)###########################################
                with open("BGRU/result_analyze_0.4_k1/TP_testcase_id_0514.txt",'a+') as ftp:
                    ftp.write(testcase[index]+'\n')
         
            results[testcase[index]] = result
    end = time.time()
    print(end - start)
        
        #保存预测的漏洞行行号
    with open(resultPath+"_result.pkl", 'wb') as f:#############################3###########3
        pickle.dump(results, f)
        
        #保存testcase到函数的预测映射
    with open(resultPath+"_dict_testcase2func.pkl", 'wb') as f:#############################
        pickle.dump(dict_testcase2func, f)
        
        #保存TP的结果
    with open("TP_index_0.4_k2_BGRU.pkl", 'wb') as f:
        pickle.dump(TP_index, f)
        
        #记录实验结果
    with open(resultPath, 'a') as fwrite:#######################################
            #实验基本信息
        fwrite.write('test_samples_num: '+ str(len(dataset)) + '\n')
        fwrite.write('train_dataset: ' + str(traindataSet_path) + '\n')#######################
        fwrite.write('test_dataset: ' + str(filename) + '\n')
        fwrite.write('model: ' + weightPath + '\n')
        #漏洞分类指标
        fwrite.write('TP:' + str(TP) + ' FP:' + str(FP) + ' FN:' + str(FN) + ' TN:' + str(TN) + '\n')
        print('TP:' ,TP , ' FP:' ,FP , ' FN:',FN , ' TN:' ,TN )
        FPR = FP / (FP + TN)
        fwrite.write('FPR: ' + str(FPR) + '\n')
        FNR = FN / (TP + FN)
        fwrite.write('FNR: ' + str(FNR) + '\n')
        accuracy = (TP + TN) / (len(dataset))
        fwrite.write('accuracy: ' + str(accuracy) + '\n')
        precision = TP / (TP + FP)
        fwrite.write('precision: ' + str(precision) + '\n')
        recall = TP / (TP + FN)
        fwrite.write('recall: ' + str(recall) + '\n')
        f_score = (2 * precision * recall) / (precision + recall)
        fwrite.write('fbeta_score: ' + str(f_score) + '\n')
        #漏洞定位指标
        fwrite.write('TP_l:' + str(TP_l) + ' FP_l:' + str(FP_l) + ' FN_l:' + str(FN_l) + ' TN:' + str(TN_l) + '\n')
        FPR_l = FP_l / (FP_l + TN_l)
        fwrite.write('FPR_location: ' + str(FPR_l) + '\n')
        FNR_l = FN_l / (TP_l + FN_l)
        fwrite.write('FNR_location: ' + str(FNR_l) + '\n')
        accuracy_l = (TP_l + TN_l) / (TP_l + FP_l + FN_l + TN_l)
        fwrite.write('accuracy_location: ' + str(accuracy_l) + '\n')
        precision_l = TP_l / (TP_l + FP_l)
        fwrite.write('precision_location: ' + str(precision_l) + '\n')
        recall_l = TP_l / (TP_l + FN_l)
        fwrite.write('recall_location: ' + str(recall_l) + '\n')
        f_score_l = (2 * precision_l * recall_l) / (precision_l + recall_l)
        fwrite.write('fbeta_score_location: ' + str(f_score_l) + '\n')
        fwrite.write('--------------------\n')
        
    print("\nf1: ",f_score)



In [47]:
#PART 3: Training
if __name__ == '__main__':
    testdataSet_path = "./data/dl_input/nvd/test"  #测试集路径
  #  realtestdataSetPath = "../data_preprocess/data/realdata/aftercut/"  
    weightPath = './model/bgru_0.4_k=1_0514.pt'  #保存模型的路径
    resultPath = "./BGRU/result/bgru_pysyft_com_0.4_k=1_0514.2"  #测试结果保存路径

    #model = local_model.send(duet)#send the model to our partners Duet Server. 

    train(remote_torch,model,weightPath)
    print("加载模型")
    if not model.is_local:
        local_model = model.get(request_block=True,name="model_download",reason="test evaluation",timeout_secs=5)
    else:
        local_model = model
    print("加载模型完成") 
    test(local_model,torch,batch_size, maxlen, vector_dim,weightPath,testdataSet_path,resultPath)
        
    print("Finished")

remote model
Train...
64
================================================================================2021-07-20 17:05:12
64
batch [1/9], Loss: 0.6463
64
batch [2/9], Loss: 0.9428
64
batch [3/9], Loss: 0.6221
64
batch [4/9], Loss: 0.6764
64
batch [5/9], Loss: 0.6666
64
batch [6/9], Loss: 0.6586
64
batch [7/9], Loss: 0.6475
64
batch [8/9], Loss: 0.6349
64
batch [9/9], Loss: 0.6324
64
batch [10/9], Loss: 0.6378
Epoch [1], Loss: 0.6765
================================================================================2021-07-20 17:06:14
64
batch [1/9], Loss: 0.6107
64
batch [2/9], Loss: 0.6089
64
batch [3/9], Loss: 0.6115
64
batch [4/9], Loss: 0.6111
64
batch [5/9], Loss: 0.6090
64
batch [6/9], Loss: 0.5985
64
batch [7/9], Loss: 0.5954
64
batch [8/9], Loss: 0.5947
64
batch [9/9], Loss: 0.5866
64


KeyboardInterrupt: 